In [1]:
%matplotlib inline

import os
import sys
genre_class_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
print("Loading Genre_Classification system from path", genre_class_path)
sys.path.append(genre_class_path)

import numpy as np
import pandas as pd
import tensorflow as tf
import json
import random

Loading Genre_Classification system from path /mnt/d/UNIV NOTES/Y4S1/DSA4199/genre_classification


2022-11-08 16:37:49.019445: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-08 16:37:49.021171: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk import download
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/kathyij74/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
dataset_wkspc = os.path.join(genre_class_path, 'workspace', 'gb_input')
data_path = os.path.join(dataset_wkspc, 'full_body_length.csv')

fullbody_df = pd.read_csv(data_path)
fullbody_df = fullbody_df.iloc[:,1:]

In [4]:
fullbody_df.head()

,body_length,subjects
0,65744,5
1,86468,5
2,86496,5
3,111279,3
4,90421,2


In [5]:
random.seed(42)

train = random.sample(range(0, len(fullbody_df)), int(len(fullbody_df)*0.7))
test = [x for x in range(0, len(fullbody_df)) if x not in train]

print(train[:20])
print(test[:20])
print(len(train))
print(len(test))

[2619, 456, 102, 3037, 1126, 1003, 914, 571, 3016, 419, 2771, 3033, 3654, 2233, 356, 2418, 1728, 130, 122, 383]
[0, 5, 7, 9, 12, 15, 17, 19, 20, 21, 27, 30, 35, 36, 37, 40, 42, 43, 44, 45]
2722
1167


In [6]:
train_df = fullbody_df.iloc[train,]
test_df = fullbody_df.iloc[test,]

print(train_df.shape)
print(test_df.shape)

(2722, 2)
(1167, 2)


In [7]:
train_df.head()

,body_length,subjects
2619,225019,3
456,98488,4
102,116806,3
3037,20268,5
1126,7758,5


In [8]:
train_df['subjects'].value_counts()

5    1231
1     378
2     318
4     296
6     258
3     241
Name: subjects, dtype: int64

In [9]:
test_df.head()

,body_length,subjects
0,65744,5
5,100690,3
7,57117,6
9,90100,3
12,113026,6


In [10]:
test_df['subjects'].value_counts()

5    511
2    151
1    145
4    131
6    116
3    113
Name: subjects, dtype: int64

In [11]:
from sklearn import svm
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = train_df['body_length'].to_numpy().reshape(-1, 1)
X = scaler.fit_transform(X)
y = train_df['subjects'].to_numpy().flatten()

clf = svm.SVC(kernel='linear')
clf.fit(X, y)

SVC(kernel='linear')

In [12]:
clf2 = svm.LinearSVC(loss='hinge', max_iter=100000)
clf2.fit(X, y)

LinearSVC(loss='hinge', max_iter=100000)

In [13]:
clf3 = svm.SVC(kernel='linear', class_weight='balanced')
clf3.fit(X, y)

SVC(class_weight='balanced', kernel='linear')

In [36]:
clf4 = svm.LinearSVC(loss='hinge', class_weight='balanced', max_iter=100000)
clf4.fit(X, y)

LinearSVC(class_weight='balanced', loss='hinge', max_iter=100000)

In [14]:
clf.coef_

array([[-2.93815306e-06],
       [-5.64989060e-01],
       [-3.50216381e-01],
       [ 1.48104208e+00],
       [-9.72972117e-07],
       [-7.98097943e-01],
       [-8.11046528e-01],
       [ 1.59020988e+00],
       [ 4.49209239e-07],
       [ 1.12399175e-06],
       [ 1.21276920e+00],
       [ 7.94230734e-01],
       [ 1.37036711e+00],
       [ 1.73163983e-07],
       [-1.66702113e+00]])

In [15]:
# make prediction

X_test = test_df['body_length'].to_numpy().reshape(-1, 1)
X_test = scaler.transform(X_test)
y_test = test_df['subjects'].to_numpy().flatten()

pred = clf.predict(X_test)

In [16]:
test_df['body_length'].to_numpy().flatten().reshape(-1, 1)

array([[ 65744],
       [100690],
       [ 57117],
       ...,
       [ 78509],
       [ 93108],
       [214222]])

In [17]:
pred

array([1, 1, 5, ..., 1, 1, 1])

In [18]:
# confusion matrix
from sklearn import metrics
import matplotlib.pyplot as plt
# import seaborn as sns

cm = metrics.confusion_matrix(y_test, pred)
cm_df = pd.DataFrame(cm, index = [1,2,3,4,5,6], columns = [1,2,3,4,5,6])
#plt.figure()
#sns.heatmap(cm_df, annot=True)
#plt.title('Confusion Matrix')
#plt.ylabel('Actual Values')
#plt.xlabel('Predicted Values')
#plt.show()

In [19]:
cm

array([[100,   0,   0,   1,  44,   0],
       [115,   0,   0,   1,  35,   0],
       [ 76,   0,   0,   9,  28,   0],
       [ 92,   0,   0,   4,  35,   0],
       [ 53,   0,   0,   1, 457,   0],
       [ 98,   0,   0,   0,  18,   0]])

In [20]:
cm.diagonal()/cm.sum(axis=1)

array([0.68965517, 0.        , 0.        , 0.03053435, 0.89432485,
       0.        ])

In [21]:
from sklearn.metrics import classification_report

print(classification_report(y_test, pred, target_names=['Adventure stories','Detective and mystery stories','Historical fiction',
                                                        'Love stories','Science fiction','Western stories'], digits=3))

                               precision    recall  f1-score   support

            Adventure stories      0.187     0.690     0.295       145
Detective and mystery stories      0.000     0.000     0.000       151
           Historical fiction      0.000     0.000     0.000       113
                 Love stories      0.250     0.031     0.054       131
              Science fiction      0.741     0.894     0.810       511
              Western stories      0.000     0.000     0.000       116

                     accuracy                          0.481      1167
                    macro avg      0.196     0.269     0.193      1167
                 weighted avg      0.376     0.481     0.398      1167



/home/kathyij74/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kathyij74/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kathyij74/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
# make prediction

X_test = test_df['body_length'].to_numpy().reshape(-1, 1)
X_test = scaler.transform(X_test)
y_test = test_df['subjects'].to_numpy().flatten()

pred2 = clf2.predict(X_test)

In [23]:
test_df['body_length'].to_numpy().flatten().reshape(-1, 1)

array([[ 65744],
       [100690],
       [ 57117],
       ...,
       [ 78509],
       [ 93108],
       [214222]])

In [24]:
pred2

array([5, 1, 5, ..., 1, 1, 1])

In [25]:
# confusion matrix
from sklearn import metrics
import matplotlib.pyplot as plt
# import seaborn as sns

cm2 = metrics.confusion_matrix(y_test, pred2)
cm_df2 = pd.DataFrame(cm2, index = [1,2,3,4,5,6], columns = [1,2,3,4,5,6])
#plt.figure()
#sns.heatmap(cm_df, annot=True)
#plt.title('Confusion Matrix')
#plt.ylabel('Actual Values')
#plt.xlabel('Predicted Values')
#plt.show()

In [26]:
cm2

array([[ 85,   0,   0,   1,  59,   0],
       [105,   0,   0,   0,  46,   0],
       [ 79,   0,   0,   4,  30,   0],
       [ 87,   0,   0,   3,  41,   0],
       [ 45,   0,   0,   0, 466,   0],
       [ 93,   0,   0,   0,  23,   0]])

In [27]:
cm2.diagonal()/cm2.sum(axis=1)

array([0.5862069 , 0.        , 0.        , 0.02290076, 0.91193738,
       0.        ])

In [28]:
from sklearn.metrics import classification_report

print(classification_report(y_test, pred2, target_names=['Adventure stories','Detective and mystery stories','Historical fiction',
                                                        'Love stories','Science fiction','Western stories'], digits=3))

                               precision    recall  f1-score   support

            Adventure stories      0.172     0.586     0.266       145
Detective and mystery stories      0.000     0.000     0.000       151
           Historical fiction      0.000     0.000     0.000       113
                 Love stories      0.375     0.023     0.043       131
              Science fiction      0.701     0.912     0.793       511
              Western stories      0.000     0.000     0.000       116

                     accuracy                          0.475      1167
                    macro avg      0.208     0.254     0.184      1167
                 weighted avg      0.370     0.475     0.385      1167



/home/kathyij74/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kathyij74/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kathyij74/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
# make prediction

X_test = test_df['body_length'].to_numpy().reshape(-1, 1)
X_test = scaler.transform(X_test)
y_test = test_df['subjects'].to_numpy().flatten()

pred3 = clf3.predict(X_test)

In [30]:
test_df['body_length'].to_numpy().flatten().reshape(-1, 1)

array([[ 65744],
       [100690],
       [ 57117],
       ...,
       [ 78509],
       [ 93108],
       [214222]])

In [31]:
pred3

array([2, 6, 2, ..., 2, 1, 3])

In [32]:
# confusion matrix
from sklearn import metrics
import matplotlib.pyplot as plt
# import seaborn as sns

cm3 = metrics.confusion_matrix(y_test, pred3)
cm_df3 = pd.DataFrame(cm3, index = [1,2,3,4,5,6], columns = [1,2,3,4,5,6])
#plt.figure()
#sns.heatmap(cm_df, annot=True)
#plt.title('Confusion Matrix')
#plt.ylabel('Actual Values')
#plt.xlabel('Predicted Values')
#plt.show()

In [33]:
cm3

array([[  9,  56,  23,   1,  21,  35],
       [ 14,  64,  11,   2,   9,  51],
       [ 12,  13,  32,   1,  18,  37],
       [ 17,  29,  29,   2,  23,  31],
       [  6,  50,   3,   0, 435,  17],
       [ 19,  38,   6,   0,   6,  47]])

In [34]:
cm3.diagonal()/cm3.sum(axis=1)

array([0.06206897, 0.42384106, 0.28318584, 0.01526718, 0.85127202,
       0.40517241])

In [35]:
from sklearn.metrics import classification_report

print(classification_report(y_test, pred3, target_names=['Adventure stories','Detective and mystery stories','Historical fiction',
                                                        'Love stories','Science fiction','Western stories'], digits=3))

                               precision    recall  f1-score   support

            Adventure stories      0.117     0.062     0.081       145
Detective and mystery stories      0.256     0.424     0.319       151
           Historical fiction      0.308     0.283     0.295       113
                 Love stories      0.333     0.015     0.029       131
              Science fiction      0.850     0.851     0.850       511
              Western stories      0.216     0.405     0.281       116

                     accuracy                          0.505      1167
                    macro avg      0.347     0.340     0.309      1167
                 weighted avg      0.508     0.505     0.484      1167



In [37]:
# make prediction

X_test = test_df['body_length'].to_numpy().reshape(-1, 1)
X_test = scaler.transform(X_test)
y_test = test_df['subjects'].to_numpy().flatten()

pred4 = clf4.predict(X_test)

In [38]:
test_df['body_length'].to_numpy().flatten().reshape(-1, 1)

array([[ 65744],
       [100690],
       [ 57117],
       ...,
       [ 78509],
       [ 93108],
       [214222]])

In [39]:
pred4

array([2, 2, 2, ..., 2, 2, 6])

In [40]:
# confusion matrix
from sklearn import metrics
import matplotlib.pyplot as plt
# import seaborn as sns

cm4 = metrics.confusion_matrix(y_test, pred4)
cm_df4 = pd.DataFrame(cm4, index = [1,2,3,4,5,6], columns = [1,2,3,4,5,6])
#plt.figure()
#sns.heatmap(cm_df, annot=True)
#plt.title('Confusion Matrix')
#plt.ylabel('Actual Values')
#plt.xlabel('Predicted Values')
#plt.show()

In [41]:
cm4

array([[ 22,  85,   0,   0,  35,   3],
       [ 13, 113,   0,   0,  23,   2],
       [ 20,  55,   1,   0,  25,  12],
       [ 22,  69,   0,   0,  31,   9],
       [  3,  58,   0,   0, 449,   1],
       [  8,  96,   0,   0,  12,   0]])

In [42]:
cm4.diagonal()/cm4.sum(axis=1)

array([0.15172414, 0.74834437, 0.00884956, 0.        , 0.87866928,
       0.        ])

In [43]:
from sklearn.metrics import classification_report

print(classification_report(y_test, pred4, target_names=['Adventure stories','Detective and mystery stories','Historical fiction',
                                                        'Love stories','Science fiction','Western stories'], digits=3))

                               precision    recall  f1-score   support

            Adventure stories      0.250     0.152     0.189       145
Detective and mystery stories      0.237     0.748     0.360       151
           Historical fiction      1.000     0.009     0.018       113
                 Love stories      0.000     0.000     0.000       131
              Science fiction      0.781     0.879     0.827       511
              Western stories      0.000     0.000     0.000       116

                     accuracy                          0.501      1167
                    macro avg      0.378     0.298     0.232      1167
                 weighted avg      0.501     0.501     0.434      1167



/home/kathyij74/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kathyij74/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kathyij74/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
